In [1]:
from kaggle_environments import make, evaluate

# BOTS WORKSHOP

All strategies are tested against the random action choice during each round of the game:

(1) Random Goblin

In [2]:
%%writefile random_goblin.py
import random
def random_goblin(observation, configuration):
    ''' each round - random action '''
    return random.randrange(0, configuration.signs)

Writing random_goblin.py


In [ ]:
evaluate(
    "rps", 
    ["random_goblin.py", "random_goblin.py"],
    configuration={"episodeSteps": 500}
)

(2) Scissors only:

In [3]:
%%writefile scissors_only.py
def scissors_only(observation, configuration):
    ''' the agent applies scissors action every round '''
    return 2

Writing scissors_only.py


In [ ]:
evaluate(
    "rps", 
    ["scissors_only.py", "random_goblin.py"],
    configuration={"episodeSteps": 500}
)

(3) Paper only:

In [4]:
%%writefile paper_only.py
def paper_only(observation, configuration):
    ''' the agent applies paper action every round '''
    return 1

Writing paper_only.py


In [ ]:
evaluate(
    "rps", 
    ["paper_only.py", "random_goblin.py"],
    configuration={"episodeSteps": 500}
)

(4) Rock only

In [5]:
%%writefile rock_only.py
def rock_only(observation, configuration):
    ''' the agent applies rock action every round '''
    return 0

Writing rock_only.py


In [ ]:
evaluate(
    "rps", 
    ["rock_only.py", "random_goblin.py"],
    configuration={"episodeSteps": 500}
)

(5) sequential bot

In [6]:
%%writefile sequential.py
import random
def sequential(observation, configuration):
    ''' the agent randomly applies first action
    then sequentially selects the next action '''
    global my_action # variable to store the agent's own action
    if observation.step == 0: # first action is random
        my_action = random.randrange(0, configuration.signs)
    else:
        # action cycling from 0 up to 2
        my_action = my_action+1 if my_action < 2 else 0
        
    return my_action

Writing sequential.py


In [ ]:
evaluate(
    "rps", 
    ["sequential.py", "random_goblin.py"],
    configuration={"episodeSteps": 500}
)

(6) copycat 

In [7]:
%%writefile copycat.py
import random
def copycat(observation, configuration):
    ''' repeats the opponent's latest action '''  
    return observation.lastOpponentAction if observation.step != 0 else random.randrange(0, configuration.signs) 

Writing copycat.py


In [ ]:
evaluate(
    "rps", 
    ["copycat.py", "random_goblin.py"],
    configuration={"episodeSteps": 500}
)

(7) bot with late reaction

In [9]:
%%writefile late_reaction.py
import random
def late_reaction(observation, configuration):
    ''' reacting to the opponent's latest action in a new round '''
    if observation.step == 0: # first action is random
        my_action = random.randrange(0, configuration.signs)
    else:
        my_action = observation.lastOpponentAction + 1 if observation.lastOpponentAction < 2 else 0
        
    return my_action

Overwriting late_reaction.py


In [ ]:
evaluate(
    "rps", 
    ["late_reaction.py", "random_goblin.py"],
    configuration={"episodeSteps": 500}
)

(8) bot that assumes the opponent never repeats the same action twice in a row

In [10]:
%%writefile never_twice.py
import random
def never_twice(observation, configuration):
    ''' the agent assumes the opponent won't repeat his action in the next round 
    thus the agent applies randomly the action againt the leftovers of the opponent'''
    if observation.step == 0: # first action is random
        my_action = random.randrange(0, configuration.signs)
    else:
        # exclude the previous opponent's action
        leftovers = [x for x in [0, 1, 2] if x != observation.lastOpponentAction]
        # random selection (guess) of the opponent's next move
        op_action = random.choice(leftovers)
        # make better action
        my_action = op_action + 1 if op_action < 2 else 0
        
    return my_action

Writing never_twice.py


In [ ]:
evaluate(
    "rps", 
    ["never_twice.py", "random_goblin.py"],
    configuration={"episodeSteps": 500}
)

(9) bot that counters the most frequent opponent's action

In [11]:
%%writefile counting_bot.py
import random
history = {0: 0, 1: 0, 2: 0} # opponent's moves counter
def counting_bot(observation, configuration):
    ''' counts the opponents previous moves and selects most frequent '''
    global history
    if observation.step == 0: # first action is random
        my_action = random.randrange(0, configuration.signs)
    else:
        # count the opponents move
        history[observation.lastOpponentAction] += 1
        # get the most frequent opponent's action 
        op_action = max(history, key=history.get)
        # counter move
        my_action = op_action + 1 if op_action < 2 else 0
        
    return my_action

Writing counting_bot.py


In [ ]:
evaluate(
    "rps", 
    ["never_twice.py", "random_goblin.py"],
    configuration={"episodeSteps": 500}
)

(10) bot with ML engine (ensemble)

In [12]:
%%writefile ensemble_bot.py
import math
import random
import numpy as np
from sklearn.ensemble import RandomForestRegressor

def get_score(left_move, right_move):
    ''' get the score of a round '''
    delta = (
        right_move - left_move
        if (left_move + right_move) % 2 == 0
        else left_move - right_move
    )
    return 0 if delta == 0 else math.copysign(1, delta)

# statistic matrix n x (my_action, op_action, score)
history = {}

def ensemble_bot(observation, configuration):
    ''' applies ensemble ML engine to predict the next move 
    the features are moves, the target is score'''
    global history 
    
    # all possible combos
    all_combos = np.array([
        [0, 0], [0, 1], [0, 2],
        [1, 0], [1, 1], [1, 2],
        [2, 0], [2, 1], [2, 2]
    ])
    
    # first 10 rounds are recorded for the train set
    if observation.step == 0:
        my_action = random.randrange(0, configuration.signs)
        op_action = -1
        history = np.array([
            my_action, op_action, get_score(my_action, op_action)
        ])
    elif 0 < observation.step <= 11:
        my_action = random.randrange(0, configuration.signs)
        op_action = observation.lastOpponentAction
        history = np.vstack([
            history,
            np.array([my_action, op_action, get_score(my_action, op_action)])
        ])
    else:
        # the zero observation dropped
        history = history[1:,:]
        # for the target the ties and losses are 0, wins are 1
        history[:, -1][history[:, -1] == -1] = 0
        # the engine is initialized and fit every round
        mind = RandomForestRegressor()
        mind.fit(history[:,:-1], history[:,-1])
        # collecting the probs of target == 1 (win) for all possible combos
        all_probs = [mind.predict([combo])[0] for combo in all_combos]
        # my action is the action with the highest win probability
        my_action = int(all_combos[np.argmax(all_probs)][0])
        # the round is added to the train dataset
        op_action = observation.lastOpponentAction
        history = np.vstack([
            history,
            np.array([my_action, op_action, get_score(my_action, op_action)])
        ])

    return my_action


Writing ensemble_bot.py


In [ ]:
env = make("rps", debug=True, configuration={"episodeSteps":500})
env.run(["random_goblin.py", "ensemble_bot.py"])
env.render(mode="ipython", width=500, height=450)

In [ ]:
evaluate(
    "rps", 
    ["ensemble_bot.py", "random_goblin.py"],
    configuration={"episodeSteps": 500}
)

(11) bot with ML engine (linear)

In [13]:
%%writefile linear_bot.py
import math
import random
import numpy as np
from sklearn.linear_model import LinearRegression

def get_score(left_move, right_move):
    ''' get the score of a round '''
    delta = (
        right_move - left_move
        if (left_move + right_move) % 2 == 0
        else left_move - right_move
    )
    return 0 if delta == 0 else math.copysign(1, delta)

# statistic matrix n x (my_action, op_action, score)
history = {}

def ensemble_bot(observation, configuration):
    ''' applies linear ML engine to predict the next move 
    the features are moves, the target is score'''
    global history 
    
    # all possible combos
    all_combos = np.array([
        [0, 0], [0, 1], [0, 2],
        [1, 0], [1, 1], [1, 2],
        [2, 0], [2, 1], [2, 2]
    ])
    
    # first 10 rounds are recorded for the train set
    if observation.step == 0:
        my_action = random.randrange(0, configuration.signs)
        op_action = -1
        history = np.array([
            my_action, op_action, get_score(my_action, op_action)
        ])
    elif 0 < observation.step <= 11:
        my_action = random.randrange(0, configuration.signs)
        op_action = observation.lastOpponentAction
        history = np.vstack([
            history,
            np.array([my_action, op_action, get_score(my_action, op_action)])
        ])
    else:
        # the zero observation dropped
        history = history[1:,:]
        # for the target the ties and losses are 0, wins are 1
        history[:, -1][history[:, -1] == -1] = 0
        # the engine is initialized and fit every round
        mind = LinearRegression()
        mind.fit(history[:,:-1], history[:,-1])
        # collecting the logits (~ probs) of target == 1 (win) for all possible combos
        all_probs = [mind.predict([combo])[0] for combo in all_combos]
        # my action is the action with the highest win probability
        my_action = int(all_combos[np.argmax(all_probs)][0])
        # the round is added to the train dataset
        op_action = observation.lastOpponentAction
        history = np.vstack([
            history,
            np.array([my_action, op_action, get_score(my_action, op_action)])
        ])

    return my_action


Writing linear_bot.py


In [ ]:
env = make("rps", debug=True, configuration={"episodeSteps":500})
env.run(["linear_bot.py", "random_goblin.py"])
env.render(mode="ipython", width=500, height=450)

In [15]:
evaluate(
    "rps", 
    ["linear_bot.py", "random_goblin.py"],
    configuration={"episodeSteps": 500}
)

[[0, 0]]

(12) bot with ML engine (neural perceptron)

In [16]:
%%writefile perceptron.py
import math
import random
import numpy as np
from sklearn.neural_network import MLPRegressor
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)

def get_score(left_move, right_move):
    ''' get the score of a round '''
    delta = (
        right_move - left_move
        if (left_move + right_move) % 2 == 0
        else left_move - right_move
    )
    return 0 if delta == 0 else math.copysign(1, delta)

# statistic matrix n x (my_action, op_action, score)
history = {}

def ensemble_bot(observation, configuration):
    ''' applies linear ML engine to predict the next move 
    the features are moves, the target is score'''
    global history 
    
    # all possible combos
    all_combos = np.array([
        [0, 0], [0, 1], [0, 2],
        [1, 0], [1, 1], [1, 2],
        [2, 0], [2, 1], [2, 2]
    ])
    
    # first 10 rounds are recorded for the train set
    if observation.step == 0:
        my_action = random.randrange(0, configuration.signs)
        op_action = -1
        history = np.array([
            my_action, op_action, get_score(my_action, op_action)
        ])
    elif 0 < observation.step <= 11:
        my_action = random.randrange(0, configuration.signs)
        op_action = observation.lastOpponentAction
        history = np.vstack([
            history,
            np.array([my_action, op_action, get_score(my_action, op_action)])
        ])
    else:
        # the zero observation dropped
        history = history[1:,:]
        # for the target the ties and losses are 0, wins are 1
        history[:, -1][history[:, -1] == -1] = 0
        # the engine is initialized and fit every round
        mind = MLPRegressor()
        mind.fit(history[:,:-1], history[:,-1])
        # collecting the logits (~ probs) of target == 1 (win) for all possible combos
        all_probs = [mind.predict([combo])[0] for combo in all_combos]
        # my action is the action with the highest win probability
        my_action = int(all_combos[np.argmax(all_probs)][0])
        # the round is added to the train dataset
        op_action = observation.lastOpponentAction
        history = np.vstack([
            history,
            np.array([my_action, op_action, get_score(my_action, op_action)])
        ])

    return my_action


Writing perceptron.py


In [ ]:
env = make("rps", debug=True, configuration={"episodeSteps":500})
env.run(["perceptron.py", "random_goblin.py"])
env.render(mode="ipython", width=500, height=450)

In [ ]:
evaluate(
    "rps", 
    ["perceptron.py", "random_goblin.py"],
    configuration={"episodeSteps": 500}
)

# BOTS COMPETITION

In [70]:
import os
import random
import numpy as np

# get the list of all participants
participants = [x for x in os.listdir() if ".py" in x]

def game_round(players):
    ''' single game of 2 players '''
    print(f"this round playing: {players}")
    game = evaluate(
        "rps", 
        playing,
        configuration={"episodeSteps": 500}
    )
    if game[0][0]==game[0][1]: # in case of tie
        winner = players
    else:
        winner = players[np.argmax(game)]
    print(f"the winner: {winner}")
    
    return winner

def series(participants):
    ''' single round for all players  '''
    print(f"this series playing: {participants}")
    winners = [] # list of winners
    while len(participants) > 1:
        players = random.sample(participants, 2) # get 2 random players
        # pop them both from the list for this round
        participants = [x for x in participants if x not in players]
        # game and t
        winners.append(game_round(players))
        # in case of uneven number of players the player without opponent 
        # goes to the next round automatically
        if len(participants) == 1:
            winners.append(participants[0])
    print(f"the winners of the series are: {winners}")
            
    return winners

def tournament(participants):
    cnt = 1
    winners = participants
    while len(winners) > 1:
        print(f"series round {cnt} started")
        winners = series(participants) 
        participants = winners
        cnt += 1
    print(f"the winner is {winners[0]}")
    
    return None

In [71]:
if __name__ == "__main__":
    tournament(participants)

series round 1 started
this series playing: ['paper_only.py', 'late_reaction.py', 'never_twice.py', 'sequential.py', 'counting_bot.py', 'copycat.py', 'random_goblin.py', 'perceptron.py', 'linear_bot.py', 'rock_only.py', 'ensemble_bot.py', 'scissors_only.py']
this round playing: ['rock_only.py', 'scissors_only.py']
the winner: rock_only.py
this round playing: ['ensemble_bot.py', 'perceptron.py']
the winner: ensemble_bot.py
this round playing: ['random_goblin.py', 'late_reaction.py']
the winner: random_goblin.py
this round playing: ['paper_only.py', 'sequential.py']
the winner: paper_only.py
this round playing: ['never_twice.py', 'counting_bot.py']
the winner: never_twice.py
this round playing: ['linear_bot.py', 'copycat.py']
the winner: linear_bot.py
the winners of the series are: ['rock_only.py', 'ensemble_bot.py', 'random_goblin.py', 'paper_only.py', 'never_twice.py', 'linear_bot.py']
series round 2 started
this series playing: ['rock_only.py', 'ensemble_bot.py', 'random_goblin.py', '